In [1]:
from torchvision import utils
from basic_fcn import *
from dataloader import *
from utils import *
import torchvision
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import time

In [2]:
train_dataset = CityScapesDataset(csv_file='train.csv')
val_dataset = CityScapesDataset(csv_file='val.csv')
test_dataset = CityScapesDataset(csv_file='test.csv')
train_loader = DataLoader(dataset=train_dataset,
                          batch_size=10,
                          num_workers=4,
                          shuffle=True)
val_loader = DataLoader(dataset=val_dataset,
                          batch_size=2,
                          num_workers=4,
                          shuffle=True)
test_loader = DataLoader(dataset=test_dataset,
                          batch_size=10,
                          num_workers=4,
                          shuffle=True)

In [3]:
#dataiter = iter(train_loader)
#images, unoHot, labels = dataiter.next()

In [4]:
def init_weights(m):
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.ConvTranspose2d):
        torch.nn.init.xavier_uniform_(m.weight.data)
        torch.nn.init.xavier_uniform_(m.bias.data.unsqueeze(0)) #add unsqueeze here 
        
epochs     = 100
criterion = nn.CrossEntropyLoss()
# Choose an appropriate loss function from https://pytorch.org/docs/stable/_modules/torch/nn/modules/loss.html
fcn_model = FCN(n_class=n_class)
fcn_model.apply(init_weights)
#fcn_model = torch.load('best_model')
optimizer = optim.Adam(fcn_model.parameters(), lr=5e-3)

In [5]:
use_gpu = torch.cuda.is_available()
if use_gpu:
    fcn_model = fcn_model.cuda()
    
def train():
    for epoch in range(epochs):
        ts = time.time()
        for iter, (X, tar, Y) in enumerate(train_loader):
            optimizer.zero_grad()

            if use_gpu:
                inputs = X.cuda() # Move your inputs onto the gpu
                labels = tar.cuda() # Move your labels onto the gpu
            else:
                inputs, labels = X, tar # Unpack variables into inputs and labels

            outputs = fcn_model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            if iter % 10 == 0:
                print("epoch{}, iter{}, loss: {}".format(epoch, iter, loss.item()))
        
        print("Finish epoch {}, time elapsed {}".format(epoch, time.time() - ts))
        #torch.save(fcn_model, 'best_model')

        #val(epoch)
        fcn_model.train()


def val(epoch):
    fcn_model.eval()
    #Complete this function - Calculate loss, accuracy and IoU for every epoch
    valLoss = 0
    valAcc = 0
    numIter = 0
    meanIOU = 0
    indexedIOU = torch.Tensor([0 for i in range(n_class)])
    print(indexedIOU)
    for iter, (X, tar, Y) in enumerate(val_loader):
        numIter +=1
        if use_gpu:
            inputs = X.cuda() # Move your inputs onto the gpu
            labels = Y.cuda()
        else:
            inputs, oneHotLabels, labels = X, tar, Y # Unpack variables into inputs and labels
        
        outputs = fcn_model(inputs)   
        valLoss += criterion(outputs, labels).item()
            
        #if use_gpu: #manage memory
        #    del labels
        #    torch.cuda.empty_cache() 
        #    oneHotLabels = tar.cuda() # Move your labels onto the gpu
            
        speakSoftlyAndCarryABigStick = F.softmax(outputs, dim = 1) #softmax along the number of class dimension
        indexes = torch.argmax(speakSoftlyAndCarryABigStick, dim = 1) #get the argmax along the channel dimension
            
        #goGoGadgetSkis = torch.zeros_like(oneHotLabels).scatter(1,indexes.unsqueeze(1), 1.0) #transform into one-hot encoding
        
        print("Unique predictions...")
        print(torch.unique(indexes))
        #print(indexes.size())
        print("unique labels")
        print(torch.unique(labels))
        #print(labels.size())
        #print(indexes)
        #print(labels)
        valAcc += pixel_acc(indexes, labels)#pixel_acc(oneHotLabels, goGoGadgetSkis_
        
        theEyesHaveIt, toBeAdded = iou(indexes, labels, n_class)
        
        meanIOU += toBeAdded
        indexedIOU = indexedIOU + theEyesHaveIt
        
        print(numIter)
            
        #manage memory again...
        if use_gpu:
            del inputs, labels, outputs, indexes, speakSoftlyAndCarryABigStick, theEyesHaveIt, toBeAdded
            torch.cuda.empty_cache() 
        
        break
            
    valAcc = valAcc/numIter
    valLoss = valLoss/numIter
    meanIOU = meanIOU/numIter
    indexedIOU = indexedIOU/numIter
    print("The validation loss is: " + str(valLoss))
    print("The pixel accuracy is: " + str(valAcc))
    print("The average IOU is: " + str(meanIOU))
    print(indexedIOU)
    print("The building IOU is: " + str(indexedIOU[11].item()))

    
def test():
    #Complete this function - Calculate accuracy and IoU 
    # Make sure to include a softmax after the output from your model
    speakSoftlyAndCarryABigStick = F.softmax(outputs, dim = 1)
    
if __name__ == "__main__":
    val(0)  # show the accuracy before training
    #train()

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
Unique predictions...
tensor([14, 16, 30, 33], device='cuda:0')
unique labels
tensor([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 11, 13, 17, 18, 19, 20, 21, 22, 23,
        24, 26, 28, 33], device='cuda:0')
1
The validation loss is: 3.521172523498535
The pixel accuracy is: 0.15056133270263672
The average IOU is: 0.0001679210690781474
tensor([   nan,    nan,    nan,    nan,    nan,    nan,    nan, 0.0000, 0.0000,
           nan,    nan, 0.0000,    nan,    nan,    nan,    nan,    nan, 0.0000,
           nan, 0.0000, 0.0000, 0.0000,    nan, 0.0000, 0.0000,    nan, 0.0000,
           nan, 0.0000,    nan,    nan,    nan,    nan, 0.0020])
The building IOU is: 0.0


34
